In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# function to query for BQ

location = "asia-northeast1"
client = bigquery.Client(location=location, project="root-beanbag-354111")
print("Client creating using default project: {}".format(client.project))

def query2df(query):
    query = query
    query_job = client.query(
        query,
        # Location must match that of the dataset(s) referenced in the query.
        location=location
    )  # API request - starts the query

    df = query_job.to_dataframe()
    return df

In [ ]:
# take table used in this project

query = """
    WITH 
        full_table AS (
            SELECT ip.img_id, ip.path, iw.wikidata_id, n.name
            FROM `root-beanbag-354111.scraping_dog_breeds_by_name.img_path` AS ip
            INNER JOIN `root-beanbag-354111.scraping_dog_breeds_by_name.img_wikidata_id` AS iw
            USING(img_id)
            INNER JOIN `root-beanbag-354111.scraping_dog_breeds_by_name.names` AS n
            USING(wikidata_id)),
        has_morethan3imgs AS (
            SELECT wikidata_id, COUNT(wikidata_id)
            FROM full_table
            GROUP BY wikidata_id
            HAVING COUNT(wikidata_id) >= 3)
    SELECT img_id, path, wikidata_id, name
    FROM full_table
    INNER JOIN has_morethan3imgs USING(wikidata_id)
"""
df = query2df(query)
df.head()

In [ ]:
def get_train_file_path(path):
    return "./" + str(Path(DATA_DIR) / Path(path))

In [ ]:
# convert to this project's image path
df['file_path'] = df['path'].apply(get_train_file_path)
df.head()